# Data Wrangling

In this notebook, we perform data cleaning, fix missing values, and add new columns with meaning values.

## Loading modules

In [8]:
import pandas as pd
import numpy as np

## Loading the data

We acquired the test score data for the [California Assessment of Student Performance and Progress (CAASPP)](https://caaspp.cde.ca.gov/). The data is available between 2015 and 2018.

* [CAASPP test scores](https://caaspp.cde.ca.gov/sb2018/ResearchFileList) 

Additional datasets are obtained in the following sites:
* [GreatSchools API](https://www.greatschools.org/api/docs/technical-overview/) 
* [Civil Rights Data Collection](https://ocrdata.ed.gov/) Teacher demographics
* [Zillow research data](https://www.zillow.com/research/data/) House prices based on zipcodes

We first load the 2018 score data.

In [21]:
df = pd.read_csv("/Users/ahrimhan/Documents/Springboard/Data/sb_ca2018_all_csv_v3/sb_ca2018_all.csv")

In [22]:
df.shape

(3269730, 32)

In [23]:
df.head()

,County Code,District Code,School Code,Filler,Test Year,Subgroup ID,Test Type,Total Tested At Entity Level,Total Tested with Scores,Grade,...,Area 1 Percentage Below Standard,Area 2 Percentage Above Standard,Area 2 Percentage Near Standard,Area 2 Percentage Below Standard,Area 3 Percentage Above Standard,Area 3 Percentage Near Standard,Area 3 Percentage Below Standard,Area 4 Percentage Above Standard,Area 4 Percentage Near Standard,Area 4 Percentage Below Standard
0,0,0,0,NaN,2018,1,B,3180554,3177403,3,...,30.66,23.84,43.30,32.85,20.89,61.25,17.86,27.60,47.71,24.68
1,0,0,0,NaN,2018,1,B,3187375,3184687,3,...,33.21,26.72,42.30,30.98,28.80,46.31,24.89,0.00,0.00,0.00
2,0,0,0,NaN,2018,1,B,3187375,3184687,4,...,39.95,21.65,44.73,33.62,24.03,43.78,32.20,0.00,0.00,0.00
3,0,0,0,NaN,2018,1,B,3180554,3177403,4,...,28.46,24.18,44.21,31.61,19.30,63.22,17.48,25.86,48.83,25.31
4,0,0,0,NaN,2018,1,B,3180554,3177403,5,...,30.76,29.20,41.42,29.38,16.52,59.74,23.73,28.29,44.32,27.39


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3269730 entries, 0 to 3269729
Data columns (total 32 columns):
County Code                          int64
District Code                        int64
School Code                          int64
Filler                               float64
Test Year                            int64
Subgroup ID                          int64
Test Type                            object
Total Tested At Entity Level         object
Total Tested with Scores             object
Grade                                int64
Test Id                              int64
CAASPP Reported Enrollment           object
Students Tested                      object
Mean Scale Score                     object
Percentage Standard Exceeded         object
Percentage Standard Met              object
Percentage Standard Met and Above    object
Percentage Standard Nearly Met       object
Percentage Standard Not Met          object
Students with Scores                 object
Area 1 Percen